In [42]:
import pandas as pd
import numpy as np
from backtesting import Backtest, Strategy
from data_storage import create_connection
from stockstats import StockDataFrame

In [43]:
connection = create_connection("../database/crypto_billionairs.db")

In [44]:
table = "ETHUSDT_1m_complete_history_long"
table_short = "ETHUSDT_1m_complete_history_short"

In [45]:
df_long = pd.read_sql_query(f'select * from {table}', connection)
df_short = pd.read_sql_query(f'select * from {table_short}', connection)

In [46]:
df_backtesting = pd.DataFrame()
if "5m" not in table:
    #df_backtesting["time"] = df_long["time"]
    df_backtesting[['Open', 'High', 'Low', 'Close', "Volume"]] = df_long[['open', 'high', 'low', 'close', 'volume']]
    #df_backtesting = df_backtesting.set_index("time")

else:
    df_backtesting[['Open', 'High', 'Low', 'Close', "Volume"]] = df_long[['open', 'high', 'low', 'close', 'volume']]

In [47]:
def calculate_osc(data):
    
   
    df_temp = pd.DataFrame()    
    stock = StockDataFrame.retype(data)
    
    df_temp["stochastic_oscillator"] = stock.get('kdjk')
    
    return df_temp["stochastic_oscillator"]

def calucalte_rsi(data):
    
    df_temp = pd.DataFrame()    
    stock = StockDataFrame.retype(data)
    
    df_temp["relative_strength_index"] = stock.get("rsi_30")
    
    return df_temp["relative_strength_index"]

def momentum(df, lag):
    return df["close"].pct_change(periods=lag)

def init_close_long_signal(df):
    
    return df_long['close_indicator']

def init_buy_signal(df):
    
    return df_long['buy_indicator']

def init_short_signal(df):
    
    return df_short['short_signals']

def init_close_short_signal(df):
    
    return df_short['short_close_signals']
    

In [48]:
class OscRsi_5m_long_short(Strategy):
   
    lag = 4
    
    def init(self):
        # compute the rsi and stochastic oscillator with stockstats and return the buy signal of the current row
        
        self.osc = self.I(calculate_osc, self.data.df)
        self.rsi = self.I(calucalte_rsi, self.data.df)
        self.buy_init = self.I(init_buy_signal, self.data.df)
        self.close_buy_init = self.I(init_close_long_signal, self.data.df)
        self.short_init = self.I(init_short_signal, self.data.df)
        self.close_short_init = self.I(init_close_short_signal, self.data.df)
        self.data.df[['Open', 'High', 'Low', 'Close', "Volume",]] = self.data.df[['open', 'high', 'low', 'close', "volume"]]
        self.returns = self.I(momentum, self.data.df, self.lag)
        self.data.df.drop(self.data.df.columns.difference(['Open', 'High', 'Low', 'Close', "Volume", "kdjk", "rsi_30"]), 1, inplace=True)
        
       
    
    def next(self):
        
        if self.buy_init[-1] == 1 and self.position.is_long is False:
            self.position.close()
            self.buy()
            
        elif self.close_buy_init[-1] == 1 and self.position.is_long:
             self.position.close()
             
             
        elif self.short_init == 1 and self.position.is_short is False:
            self.position.close()
            self.sell()


In [49]:
bt = Backtest(df_backtesting, OscRsi_5m_long_short, cash=10_000, commission=.001)
stats = bt.run()
stats

<ipython-input-49-8d03f2706277>:1: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df_backtesting, OscRsi_5m_long_short, cash=10_000, commission=.001)


Start                                       0
End                               2.74946e+06
Duration                          2.74946e+06
Exposure Time [%]                      32.191
Equity Final [$]                      44169.6
Equity Peak [$]                        176244
Return [%]                            341.696
Buy & Hold Return [%]                 29047.7
Return (Ann.) [%]                           0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                                0
Max. Drawdown [%]                     -93.838
Avg. Drawdown [%]                    -1.80242
Max. Drawdown Duration            2.20442e+06
Avg. Drawdown Duration                 4716.6
# Trades                                  691
Win Rate [%]                          48.4805
Best Trade [%]                        45.1384
Worst Trade [%]                       -25.282
Avg. Trade [%]                    

In [50]:
bt.plot()

Row(id='1797', ...)

In [51]:
# %%time 
# stats = bt.optimize(lag=[12, 24, 48, 96, 120, 144, 168],
#                     sell_ = np.arange(60, 90, 3).tolist(),#sell_threshold = np.arange(0, -0.15, -0.01).tolist(),
#                     rsi_lower_bound = np.arange(10, 40, 3).tolist(),
#                     rsi_upper_bound = np.arange(12, 50, 3).tolist(),
#                     #osc_lower_bound = np.arange(10, 90, 5).tolist(),
#                     #osc_upper_bound = np.arange(10, 100, 5).tolist(),
#                     #maximize='Equity Final [$]')#,
#                     constraint= lambda param: param.rsi_lower_bound < param.rsi_upper_bound)
# stats

In [52]:
stats._strategy

<Strategy OscRsi_5m_long_short>

In [53]:
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,868,3057,5073,11.515504,11.8340,276.454528,0.027658,3057,5073,2016
1,1108,7268,9284,9.271262,8.1986,-1188.509496,-0.115698,7268,9284,2016
2,1107,13572,15588,8.207399,9.7370,1693.268086,0.186369,13572,15588,2016
3,778,34960,36976,13.841828,16.9100,2387.037816,0.221659,34960,36976,2016
4,780,38118,40134,16.876860,20.6230,2921.989200,0.221969,38118,40134,2016
...,...,...,...,...,...,...,...,...,...,...
686,16,2715151,2715221,2717.544830,2721.7900,67.922720,0.001562,2715151,2715221,70
687,17,2720252,2722268,2611.889280,2528.3200,-1420.677760,-0.031996,2720252,2722268,2016
688,16,2722369,2724055,2589.446860,2524.5300,-1038.669760,-0.025070,2722369,2724055,1686
689,16,2724703,2726719,2654.752100,2767.3800,1802.046400,0.042425,2724703,2726719,2016


In [54]:
trades = pd.DataFrame(stats['_trades'])

In [55]:
trades.to_excel(f"trades_{table}_short.xlsx")